In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import cufflinks as cf
cf.go_offline()
%matplotlib inline

In [ ]:
#Special Imports
import inflect as inf
import re
import requests 
import bs4
from re import sub
from collections import Counter

In [ ]:
###CAMELCASE FUNCTION###
def camel_case(s):
    #not true camelcase but adjusted for queries 
    
  s = sub(r"(_|-)+", " ", s).title().replace(" ", "")
  return ''.join([s[0].upper(), s[1:]])


In [ ]:
#Read In DF
df = pd.read_csv('BigML_Dataset_5f50a62c2fb31c516d000176.csv')

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
plt.figure(figsize = (12,10))
sns.heatmap(df.isnull(), yticklabels = False, cbar = False, cmap = 'viridis')

In [ ]:
#Need to split the genre string into a list
df['genre'] = df['genre'].apply(lambda string: string.split('|'))


In [ ]:
for i in df.columns:
    print(i)

In [ ]:
#Pick the Oscars columns, and some of the more reliable columns for the other movies. 
oscar = df[['year', 'movie', 'certificate', 'duration', 'metascore', 'rate', 'user_reviews',
 'genre', 'gross', 'release_date', 'Oscar_Best_Picture_won',
 'Oscar_Best_Picture_nominated', 'Oscar_Best_Director_won',
'Oscar_Best_Director_nominated', 'Oscar_Best_Actor_won' ,
'Oscar_Best_Actor_nominated', 'Oscar_Best_Actress_won',
 'Oscar_Best_Actress_nominated',
'Oscar_Best_Supporting_Actor_won',
'Oscar_Best_Supporting_Actor_nominated',
'Oscar_Best_Supporting_Actress_won',
'Oscar_Best_Supporting_Actress_nominated',
'Oscar_Best_AdaScreen_won',
'Oscar_Best_AdaScreen_nominated',
'Oscar_Best_OriScreen_won',
'Oscar_Best_OriScreen_nominated',
'Oscar_nominated',
'Oscar_nominated_categories',
'release_date.year',
'release_date.month',
'release_date.day-of-month',
'release_date.day-of-week']].copy()


In [ ]:
oscar.info()

In [ ]:
oscar.describe()

In [ ]:
oscar.sort_values(['year', 'movie'])

In [ ]:
oscar = oscar.drop(['release_date.year','release_date.month','release_date.day-of-month', 'release_date.day-of-week'], axis = 1)

In [ ]:
oscar.head()

In [ ]:
oscar['Nominated Categories'] = oscar['Oscar_nominated_categories'].fillna('').apply(lambda line: line.split('|'))

In [ ]:
oscar.drop('Oscar_nominated_categories', axis= 1, inplace = True)

In [ ]:
oscar.head()

In [ ]:
#We map the strings No, Yes into numbers for easy evaluation
bool_dict = {'No': 0 , 'Yes': 1}

In [ ]:
oscar['Oscar_Best_Picture_won'] = oscar['Oscar_Best_Picture_won'].map(bool_dict)
oscar['Oscar_Best_Picture_nominated'] = oscar[ 'Oscar_Best_Picture_nominated'].map(bool_dict)
oscar['Oscar_Best_Director_won'] = oscar['Oscar_Best_Director_won'].map(bool_dict)
oscar['Oscar_Best_Director_nominated'] = oscar['Oscar_Best_Director_nominated'].map(bool_dict)
oscar['Oscar_Best_Actor_won'] = oscar['Oscar_Best_Actor_won'].map(bool_dict)
oscar['Oscar_Best_Actor_nominated'] = oscar['Oscar_Best_Actor_nominated'].map(bool_dict)
oscar['Oscar_Best_Actress_won'] = oscar['Oscar_Best_Actress_won'].map(bool_dict)
oscar['Oscar_Best_Actress_nominated'] = oscar['Oscar_Best_Actress_nominated'].map(bool_dict)
oscar['Oscar_Best_Supporting_Actor_won'] = oscar['Oscar_Best_Supporting_Actor_won'].map(bool_dict)
oscar['Oscar_Best_Supporting_Actor_nominated'] = oscar['Oscar_Best_Supporting_Actor_nominated'].map(bool_dict)
oscar['Oscar_Best_Supporting_Actress_won'] = oscar['Oscar_Best_Supporting_Actress_won'].map(bool_dict)
oscar['Oscar_Best_Supporting_Actress_nominated'] = oscar['Oscar_Best_Supporting_Actress_nominated'].map(bool_dict)
oscar['Oscar_Best_AdaScreen_won'] = oscar['Oscar_Best_AdaScreen_won'].map(bool_dict)
oscar['Oscar_Best_AdaScreen_nominated'] = oscar['Oscar_Best_AdaScreen_nominated'].map(bool_dict)
oscar['Oscar_Best_OriScreen_won'] = oscar['Oscar_Best_OriScreen_won'].map(bool_dict)
oscar['Oscar_Best_OriScreen_nominated'] = oscar['Oscar_Best_OriScreen_nominated'].map(bool_dict)

In [ ]:
#We process the release date into a datetime object,
#Make OscarsWon column, and Oscar Win Ratio
#Check gross, metascore, and Oscar Win Ratio

In [ ]:
oscar.head()

In [ ]:
oscar['release_date'].apply(pd.to_datetime) #Here's where I see missing release dates
#Let's do a heatmap to see if theres more missing stuff

In [ ]:
plt.figure(figsize = (12,10))
sns.heatmap(oscar.isnull(), yticklabels = False, cbar = False, cmap = 'viridis')

In [ ]:
oscar[oscar['certificate'].isna()]

In [ ]:
#Manually impute missing data
#Having manually checked these:
#All the NA's in  the certificate column, are actually NR--Not Rated movies

oscar['certificate'] = oscar['certificate'].fillna('Not Rated')



In [ ]:
oscar['certificate'].value_counts() #We need to move the unrated ones to Not Rated

In [ ]:
# doesnt work oscar[oscar['certificate']=='Unrated'].at[:,'certificate'] = 'Not Rated'

In [ ]:
# doesnt work oscar['certificate'].apply(lambda line: line.replace('Unrated', 'Not Rated'))

In [ ]:
oscar[oscar['certificate'] == 'Unrated']

In [ ]:
oscar.loc[[130,577,781,836], 'certificate'] = 'Not Rated'

In [ ]:
oscar['certificate'].value_counts()

In [ ]:
oscar[oscar['certificate'] == 'TV-MA']

In [ ]:
plt.figure(figsize = (12,10))
sns.heatmap(oscar.isnull(), yticklabels = False, cbar = False, cmap = 'viridis')

In [ ]:
oscar[oscar['metascore'].isna()]

In [ ]:
oscar.loc[[88, 272, 348,
           380, 450, 474,
          577, 652, 669,
           700,701,857,
           962, 992],'metascore'] = [None ,None, 9, 
                                    71, None, 91, 
                                    None, None, None,
                                    None, None, 73, 
                                    87, None] 

In [ ]:
oscar[oscar['metascore'].isna()] #Genuinely no metascore for these

In [ ]:
plt.figure(figsize = (12,10))
sns.heatmap(oscar.isnull(), yticklabels = False, cbar = False, cmap = 'viridis')

In [ ]:
oscar[oscar['user_reviews'].isna()]

In [ ]:
oscar.loc[oscar[oscar['user_reviews'].isna()].index, 'user_reviews'] = [354, 202, 1238, 249, 273 , 805, 255]

In [ ]:
oscar[oscar['user_reviews'].isna()]

In [ ]:
plt.figure(figsize = (12,10))
sns.heatmap(oscar.isnull(), yticklabels = False, cbar = False, cmap = 'viridis')

In [ ]:
oscar[oscar['gross'].isna()]

In [ ]:
oscar.loc[oscar[oscar['gross'].isna()].index,'gross'] = [1724806, 1058905, 281219,
                                                  7372769,  4988181, 246439, 
                                                  1669947, 14924998, 31126421,
                                                  46749646, 886407, 3470487,
                                                 34854990, 334933831, 85955, 
                                                 4439481, 614444, 11430025,
                                                 6170043, 12748594, 3169553,
                                                 4086096, 278454417, 8236 ]

In [ ]:
# This does not work to index and replace , do you know why? 
# oscar['gross'].isna().index)

In [ ]:
oscar[oscar['gross'].isna()]

In [ ]:
plt.figure(figsize = (12,10))
sns.heatmap(oscar.isnull(), yticklabels = False, cbar = False, cmap = 'viridis')

In [ ]:
oscar[oscar['release_date'].isna()]

In [ ]:
oscar.loc[oscar[oscar['release_date'].isna()].index,'release_date'] = ['2017-01-21', '2017-11-17',
                                                                       '2017-11-10', '2017-11-10',
                                                                      '2017-10-13', '2017-10-05',
                                                                       '2018-01-19', '2017-12-25',
                                                                       '2017-11-17']

In [ ]:
oscar[oscar['release_date'].isna()]

In [ ]:
plt.figure(figsize = (12,10))
sns.heatmap(oscar.isnull(), yticklabels = False, cbar = False, cmap = 'viridis')

In [ ]:
#This is the best we can do.
#We can try to impute the metascore based on user_rating (rate)
#But we'll save that for another project. 



In [ ]:
#Convert release date to date_time
oscar['release_date'] = oscar['release_date'].apply(pd.to_datetime) 

In [ ]:
oscar['release_date'].apply(lambda stamp: stamp.month)

In [ ]:
oscar['release_year'] = oscar['release_date'].apply(lambda stamp: stamp.year)  

In [ ]:
oscar['release_month'] = oscar['release_date'].apply(lambda stamp: stamp.month)  

In [ ]:
oscar['release_day'] = oscar['release_date'].apply(lambda stamp: stamp.day_of_week)  

In [ ]:
d_dict = {0: 'Sunday', 1: 'Monday', 2: 'Tuesday',
3: 'Wednesday', 4: 'Thursday', 5:'Friday', 6:'Saturday'}

In [ ]:
oscar['release_day_of_week'] = oscar['release_day'].map(d_dict)

In [ ]:
oscar.columns

In [ ]:
X = oscar[['year', 'movie', 'certificate', 'duration', 'metascore', 'rate',
       'user_reviews', 'genre', 'gross', 'release_date','release_year', 'release_month', 'release_day',
       'release_day_of_week', 'Oscar_nominated']]

In [ ]:
X.head()

In [ ]:
X['Nominated'] = X['Oscar_nominated'] > 0

In [ ]:
#Make nominated to win ratio for movies

In [ ]:
 s = oscar['Oscar_Best_Picture_won']|oscar['Oscar_Best_Director_won'] | oscar['Oscar_Best_Actor_won'] | oscar['Oscar_Best_Actress_won']| oscar['Oscar_Best_Supporting_Actor_won'] | oscar['Oscar_Best_Supporting_Actor_won'] | oscar['Oscar_Best_Supporting_Actress_won'] | oscar['Oscar_Best_AdaScreen_won'] |oscar['Oscar_Best_OriScreen_won']

In [ ]:
X['Won'] = s

In [ ]:
X['Won_Nominated'] = X['Won'] + X['Nominated']

In [ ]:
X['Num_Won'] = oscar['Oscar_Best_Picture_won'] + oscar['Oscar_Best_Director_won'] + oscar['Oscar_Best_Actor_won'] + oscar['Oscar_Best_Actress_won'] + oscar['Oscar_Best_Supporting_Actor_won'] + oscar['Oscar_Best_Supporting_Actor_won'] + oscar['Oscar_Best_Supporting_Actress_won'] + oscar['Oscar_Best_AdaScreen_won'] + oscar['Oscar_Best_OriScreen_won']

In [ ]:
X['Win_Ratio'] = (X['Num_Won']/X['Oscar_nominated']).fillna(0)

In [ ]:
sns.pairplot(X.drop(['release_day_of_week',	'Oscar_nominated',	'Nominated',	'Num_Won', 'Won'], axis =1), hue ='Won_Nominated',
            palette = 'Set1')